In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from math import sqrt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_data= pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

**K-Neighbors**
It does not do well with big data

**Decision Tree Regressor**
It is beneficial because the data does not need to be scaled.

**Gradient Boosting**
This decision tree model also doesn't care about data scaling. It can counter overfitting by adjusting parameters.

In [3]:
#Please make sure the import the related libraries as well.  
#Below is the code for the regression_ function for you to copy/paste (please indent the code accordingly) :

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error


def regression_(x,y,a, i):
    #I include the regression models I will use.
    L=LinearRegression()
    R=Ridge(alpha=a)
    Lass=Lasso(alpha=0.0001, max_iter=100000)
    E=ElasticNet(alpha=a)
    ETR=ExtraTreeRegressor(max_depth= i)
    GBR=GradientBoostingRegressor(alpha=a, n_estimators=1200, learning_rate=0.1)
    XGBC=XGBRegressor(max_depth=i)
    dt=DecisionTreeRegressor(max_depth=i)
    kn=KNeighborsRegressor(n_neighbors=3)
    
    #I separate the data as train and test. (20% test)
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
    
    #I put my models and model names in a list.
    algos=[L,R,Lass,E,ETR,GBR,XGBC,dt,kn]
    algo_names=['Linear','Ridge','Lasso(alpha=0.0001)','ElasticNet','Extra Tree','Gradient Boosting(n=1200, LR=0.1)','XGradientBoosting','DecisionTree','KNeighbors(n_neighbors=3)']
    
    #I create empty lists for error rates and success rates, which I will fill in later.
    r2Score = []
    rmse = []
    mae = []
    
    #I am creating a dataframe as I want to see all results as a table.
    #Its columns will be 'R2_score', 'RMSE', 'MAE'. The indexes will take from the string I created for the model names.
    result=pd.DataFrame(columns=['R2_score','RMSE','MAE'],index=algo_names)
    
    #I run the regression algorithms for each model and find the results of r2_score, mean_absolute_error and mean_squared_error.
    #And I add these results I found to the empty lists I created above with the append () function.
    for algo in algos:
        p=algo.fit(x_train,y_train).predict(x_test)
        r2Score.append(r2_score(y_test,p))
        rmse.append(mean_squared_error(y_test,p)**.5)
        mae.append(mean_absolute_error(y_test,p))
        
    #I fill the columns in the table with these lists.
    result['R2_score']=r2Score
    result.RMSE=rmse
    result.MAE=mae

    #I sort my result table by r2_score value and return it.
    return result.sort_values('R2_score',ascending=False)

In [4]:
# Selecting features. This is a basic model, so we'll use only numerical features for simplicity,
# and explicitly exclude the 'Id' column as it's not a predictive feature.
features = train_data.select_dtypes(include=['int64', 'float64']).columns.drop(['SalePrice', 'Id'])
X = train_data[features]
y = train_data['SalePrice']
X_test = test_data[features]

# Data preprocessing
# Impute missing values
imputer = SimpleImputer()
X = imputer.fit_transform(X)
X_test = imputer.transform(X_test)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# Model training
model = GradientBoostingRegressor(alpha=0.1, n_estimators=1200, learning_rate=0.1).fit(X, y)
#model.fit(X, y)

# Predictions
predictions = model.predict(X_test)

# Create submission file
submission = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created.")

Submission file 'submission.csv' created.


In [5]:
# Run the regression_ fx to see which algorithm does the best
regression_(X, y, 0.1, 1)

,R2_score,RMSE,MAE
"Gradient Boosting(n=1200, LR=0.1)",0.896410,28188.158534,17223.819827
XGradientBoosting,0.870883,31470.127555,20209.497646
Linear,0.823164,36829.169083,22981.338523
Ridge,0.823151,36830.506980,22979.594315
Lasso(alpha=0.0001),0.823151,36830.593777,22980.495906
ElasticNet,0.822723,36875.104420,22611.668095
KNeighbors(n_neighbors=3),0.792917,39854.642961,23143.476027
DecisionTree,0.446135,65179.178177,46155.509182
Extra Tree,0.307484,72882.277667,51045.814174
